In [1]:
import json
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Preprocess a single text string
def preprocess_text(text):
    # Tokenize data
    tokens = nltk.word_tokenize(text)
    # print (f"tokens {tokens}")
    # Lowercase all words
    tokens = [word.lower() for word in tokens]
    # print (f"lower tokens {tokens}")
    
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    # print (f"stop-words tokens {tokens}")
    
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word, pos= 'v') for word in tokens]
    tokens = [lemmatizer.lemmatize(word, pos= 'a') for word in tokens]
    # tokens = [lemmatizer.lemmatize(word, pos= 'n') for word in tokens]
    tokens = [lemmatizer.lemmatize(word, pos= 'r') for word in tokens]
    # print (f"(end) lemmatizer tokens {tokens}")
    # print("")
    return tokens

# Read JSON data from a file
with open('newData.json', 'r') as json_file:
    json_data = json.load(json_file)

# Preprocess the "patterns" and "responses" values in each dictionary
for item in json_data:
    item["patterns"] = [preprocess_text(pattern) for pattern in item["patterns"]]
    item["responses"] = [preprocess_text(response) for response in item["responses"]]

# Now, json_data contains preprocessed "patterns" and "responses"


In [2]:
json_data

[{'tag': 'product_warranty',
  'patterns': [['product', 'come', 'warranty']],
  'responses': [['yes', 'product', 'come', 'one-year', 'warranty'],
   ['product', 'come', 'warranty'],
   ['warranty', 'product', 'one', 'year'],
   ['please', 'contact', 'us', 'information', 'warranty', 'product']]},
 {'tag': 'product_return',
  'patterns': [['return', 'item', "n't", 'like']],
  'responses': [['yes', 'return', 'item', 'within', '0', 'days', 'purchase'],
   ['return', 'item'],
   ['please', 'contact', 'us', 'information', 'return', 'policy', 'product']]},
 {'tag': 'product_free_shipping',
  'patterns': [['offer', 'free', 'ship']],
  'responses': [['yes', 'offer', 'free', 'ship', 'order', '0'],
   ['offer', 'free', 'ship']]},
 {'tag': 'product_discount',
  'patterns': [['discount', 'code', 'available']],
  'responses': [['yes', 'discount', 'available', '10', 'save', 'ratio'],
   ['discount', 'code', 'available', 'time'],
   ['please', 'contact', 'us', 'information', 'discount', 'cod']]},
 {'t

In [17]:
with open('newData.json', 'r') as json_file:
    theData = json.load(json_file)

import pandas as pd
df = pd.DataFrame(theData)
df['responses'][0]

['Yes, this product comes with a one-year warranty.',
 'No, this product does not come with a warranty.',
 'The warranty for this product is one year.',
 'Please contact us for more information about the warranty for this product.']

## Classification using DT

In [5]:
def restructure_data(data):
    structured_data = []

    for item in data:
        tag = item['tag']
        patterns = item['patterns']

        for pattern in patterns:
            feature_dict = {word: True for word in pattern}
            structured_data.append((feature_dict, tag))

    return structured_data


In [6]:
data = restructure_data(json_data)

## Classification using Naive Bayes

In [8]:
def train_using_naive_bayes (training_data):
    classifier = nltk. NaiveBayesClassifier.train(data)
    classifier_name = type(classifier).__name__
    return classifier, classifier_name

classifier, classifier_name = train_using_naive_bayes (data)
print(len(classifier.most_informative_features())) 
classifier.show_most_informative_features()

100
Most Informative Features
                   order = True           order_ : shippi =      2.8 : 1.0
                  arrive = True           parcel : shippi =      2.5 : 1.0
               available = True           produc : shippi =      2.5 : 1.0
                    date = True           change : shippi =      2.5 : 1.0
                    long = True           order_ : shippi =      2.5 : 1.0
                 options = True           produc : shippi =      2.5 : 1.0
                    ship = True           expedi : shippi =      2.5 : 1.0
                  accept = None           shippi : paymen =      2.4 : 1.0
                 methods = True           paymen : paymen =      2.0 : 1.0
                 payment = True           paymen : paymen =      2.0 : 1.0


In [9]:
import random
import nltk

# Train the Naive Bayes classifier
classifier = nltk.NaiveBayesClassifier.train(data)

# Define a function to preprocess text (similar to your preprocess_text function)
def preprocess_text(text):
    # Tokenize data
    tokens = nltk.word_tokenize(text)

    # Lowercase all words
    tokens = [word.lower() for word in tokens]

    # Remove stopwords and punctuation
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]

    # Lemmatize words
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

# Define a function to preprocess a single text string and convert it to a feature dictionary
def preprocess_text_to_features(text):
    preprocessed_text = preprocess_text(text)
    feature_dict = {word: True for word in preprocessed_text}
    return feature_dict

# Define a function to get a response for a user's question using a Naive Bayes classifier
def get_response_naive_bayes(question, classifier, data):
    # Preprocess the question and convert it to a feature dictionary
    feature_dict = preprocess_text_to_features(question)
    
    # Use the classifier to classify the question
    predicted_tag = classifier.classify(feature_dict)
    
    # Retrieve the corresponding response for the predicted tag
    for item in data:
        if item['tag'] == predicted_tag:
            responses = item['responses']
            # Return a random response from the list of responses as a formatted sentence
            return random.choice(responses)

with open('newData.json', 'r') as json_file:
    Alldata = json.load(json_file)

Chatbot: There are a few reasons why the images on our site might not be loading properly.


In [28]:
test_list = ["Is there a warranty for this product?" ,"What is the return policy if I'm not satisfied with my purchase?" ,"Do you provide free shipping?" ,"Are there any discount codes available?" ,"When can I expect my order to arrive?" ,"What is the estimated delivery date?" ,"Are payments via Google Wallet accepted?" ,"Can I use Apple Cash for payment?" ,"What payment methods are accepted?" ,"What are the key features of this product?" ,"When is my order expected to be delivered?" ,"Where can I find information on the return policy?" ,"How much does this product cost?" ,"How can I contact customer service?" ,"What color options are available for this product?" ,"What payment methods are accepted?" ,"What are the available shipping options?" ,"What are the return policies?" ,"What is your opinion on this product?" ,"What are the reasons for returning this item?" ,"Can you recommend a smartphone with a good camera and long battery life?" ,"What are the features of the latest model of {product_name}?" ,"Can you provide a list of the best-selling products in {category}?" ,"Are there different color or size options available for this product?" ,"What is the price range for {product_name} products?" ,"What are the dimensions and weight of this item?" ,"Can you show me pictures of both the front and back of the dress?" ,"Do you offer engraving or customization options?" ,"Can you provide an update on the status of my recent order?" ,"When can I expect my package to be delivered?" ,"How can I track my shipment using the tracking number?" ,"Is it possible to change the shipping address for my order?" ,"Can you confirm whether my payment for the order was successful?" ,"How do I determine the estimated arrival date of my parcel?" ,"Why was my order delayed, and how much longer will it take?" ,"How can I initiate a return for a product I purchased?" ,"What is the refund process if I'm not satisfied with my purchase?" ,"Can I exchange a product for a different size or color?" ,"How long does it take to process a refund after receiving the return?" ,"What is the policy for returning products purchased on sale?" ,"How can I apply a discount code during checkout?" ,"What should I do if my payment is declined during checkout?" ,"Can I split the payment between two different cards?" ,"Is it possible to save my payment information for future purchases?" ,"How many installments can I split my purchase into?" ,"May I submit an alternative payment method after the order is processed?" ,"Will foreign currency transactions incur additional fees?" ,"How can I create an account on your website?" ,"How do I update my shipping address in my account settings?" ,"I forgot my password. How can I reset it?" ,"Can I change the email address associated with my account?" ,"How can I delete my account from your website?" ,"What expedited shipping options are available?" ,"Will taxes be added during checkout or upon delivery?" ,"Which carrier will deliver my items?" ,"Do you offer signature confirmation options?" ,"Can I modify the delivery date or timeframe for my order?" ,"Are there any free gift offers based on the order total?" ,"Can I use multiple promo codes during checkout?" ,"Are there exclusive Black Friday deals for specific categories?" ,"If I purchase two items today, will I get a discount on my next purchase?" ,"Are there physical store locations where I can pick up items?" ,"How can I track returns, replacements, or exchanges online?" ,"Can I speak with a specialist if I encounter technical issues on your website?" ,"Can you email me copies of past receipts or correspondence?" ,"How can I reset my password if I've forgotten it?" ,"I'm having trouble logging into my account. What's the issue?" ,"I encounter errors while filling out forms. Can you assist me?" ,"Clicking 'Zoom' doesn't work. Can you help resolve this issue?" ,"Images on your page aren't loading correctly."]
print(test_list)

['Is there a warranty for this product?', "What is the return policy if I'm not satisfied with my purchase?", 'Do you provide free shipping?', 'Are there any discount codes available?', 'When can I expect my order to arrive?', 'What is the estimated delivery date?', 'Are payments via Google Wallet accepted?', 'Can I use Apple Cash for payment?', 'What payment methods are accepted?', 'What are the key features of this product?', 'When is my order expected to be delivered?', 'Where can I find information on the return policy?', 'How much does this product cost?', 'How can I contact customer service?', 'What color options are available for this product?', 'What payment methods are accepted?', 'What are the available shipping options?', 'What are the return policies?', 'What is your opinion on this product?', 'What are the reasons for returning this item?', 'Can you recommend a smartphone with a good camera and long battery life?', 'What are the features of the latest model of {product_nam

In [ ]:
# Testing ( I have some problems here to find the accuracy of the model)
number = 0
c = 0
for x in test_list:
    response = get_response_naive_bayes(x, classifier, Alldata)
    if c < len(df):
        if response in df['responses'][c]:
            number += 1
        else:
            print(c)
            print(f"response :{response}")
            print(f"all response :{df['responses'][c]}")
        c += 1
print(number)


['Is there a warranty for this product?', "What is the return policy if I'm not satisfied with my purchase?", 'Do you provide free shipping?', 'Are there any discount codes available?', 'When can I expect my order to arrive?', 'What is the estimated delivery date?', 'Are payments via Google Wallet accepted?', 'Can I use Apple Cash for payment?', 'What payment methods are accepted?', 'What are the key features of this product?', 'When is my order expected to be delivered?', 'Where can I find information on the return policy?', 'How much does this product cost?', 'How can I contact customer service?', 'What color options are available for this product?', 'What payment methods are accepted?', 'What are the available shipping options?', 'What are the return policies?', 'What is your opinion on this product?', 'What are the reasons for returning this item?', 'Can you recommend a smartphone with a good camera and long battery life?', 'What are the features of the latest model of {product_nam